In [1]:
import os
import sys
root_dir = os.path.abspath('../')
sys.path.append(root_dir)

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# --- Stunning palette: Viridis or Plasma ---
palette = sns.color_palette("viridis", 8)

sns.set_theme(
    style="whitegrid",
    font="sans-serif",
    rc={
        "font.size": 13,
        "axes.titlesize": 16,
        "axes.titleweight": "bold",
        "axes.labelsize": 13,
        "axes.labelweight": "semibold",
        "axes.edgecolor": "#2F2F2F",
        "axes.linewidth": 0.8,
        "grid.color": "#CCCCCC",
        "grid.linewidth": 0.6,
        "grid.alpha": 0.35,
        "figure.facecolor": "#FAFAFA",
        "axes.facecolor": "#FFFFFF",
        "axes.prop_cycle": plt.cycler("color", palette),
    }
)

In [2]:
from config import LAVARConfig
from dataset import RollingXYDataset, RollingXYDatasetWithY0
from scalers import StandardScalerTorch
from models import LAVAR
from train_stage1 import stage1_train_lavar
from train_stage2 import stage2_train_supply_density_split

import traceback

import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

/Users/goonzard/Developer/ds-202512-lavar/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Engineered data
lavar_data1 = pd.read_parquet(
    os.path.join(root_dir, "example/lavar_daily1.parquet")
)  # demographics, diagnosis code

lavar_data2 = pd.read_parquet(
    os.path.join(root_dir, "example/lavar_daily2.parquet")
)  # demographics, ATC code

# Resample, daily-wise
full_range = pd.date_range(lavar_data1.index.min(), lavar_data1.index.max(), freq='D')
lavar_data1 = lavar_data1.reindex(full_range)
lavar_data2 = lavar_data2.reindex(full_range)

lavar_data1 = lavar_data1.fillna(0)
lavar_data2 = lavar_data2.fillna(0)

common = [i for i in lavar_data1.columns if i in lavar_data2.columns]
col1 = [i for i in lavar_data1.columns if i not in common]
col2 = [i for i in lavar_data2.columns if i not in common]

non_supply_df = lavar_data1[common + col1]
non_supply_cols = common + col1
supply_df = lavar_data2[col2]
supply_cols = col2

X = torch.tensor(non_supply_df.values, dtype=torch.float32)  # (T, Dx)
y = torch.tensor(supply_df.values, dtype=torch.float32)      # (T, Dy)

# (torch.Size([2556, 40]), torch.Size([2556, 36]))
# T = 2556, Dx = 40, Dy = 36
X.shape, y.shape  

(torch.Size([2556, 40]), torch.Size([2556, 36]))

In [4]:
cfg = LAVARConfig(
    device="mps",
    horizon=14,

    # Density split thresholds (computed on TRAIN only):
    dense_nonzero_rate_thr=0.70,
    ultra_nonzero_rate_thr=0.005,

    latent_dim=8,
    encoder_hidden=[32, 16],
    decoder_hidden=[16, 32],
    supply_hidden=[16],

    batch_size=256,
    epochs_lavar=300,
    epochs_supply=300,
    train_days=365 * 2,
)

In [5]:
T = X.shape[0]
INPUT_DIM = X.shape[1]
min_train = cfg.train_days
H = cfg.horizon
step = 7

print("Total samples: ", T)
print("Input dimension: ", INPUT_DIM)
print("Training samples: ", min_train)  # 3 years
print("Horizon: ", H)  # 14 days
print("Step: ", step)  # 7 days, Weekly rolling

Total samples:  2556
Input dimension:  40
Training samples:  730
Horizon:  14
Step:  7


In [ ]:
fold_ends = []
t = min_train
while t + H <= T:
    fold_ends.append(t)
    t += step

all_fold_errors = []
all_fold_rmse_by_target = []  # list of (Dy,) per fold
all_pred_times, all_pred_values, all_true_values = [], [], []

n_folds = len(fold_ends)
for fold_id, t_end in enumerate(tqdm(fold_ends, desc="Rolling folds", unit="fold"), start=1):
    try:
        tqdm.write(f"\n=== Fold {fold_id}/{n_folds} | train until day {t_end} ===")

        X_train_raw = X[:t_end]
        y_train_raw = y[:t_end]

        # give buffer so rolling windows are valid
        X_val_raw = X[t_end - (cfg.dyn_p + cfg.horizon) : t_end + cfg.horizon]
        y_val_raw = y[t_end - (cfg.dyn_p + cfg.horizon) : t_end + cfg.horizon]

        x_scaler = StandardScalerTorch().fit(X_train_raw)
        X_train = x_scaler.transform(X_train_raw)
        X_val = x_scaler.transform(X_val_raw)

        # Use raw y for supervision in this notebook.
        y_scaler = None
        y_train = y_train_raw
        y_val = y_val_raw

        # Stage 1 uses X only (hospital-independent latent model)
        stage1_train_ds = RollingXYDataset(x=X_train, y=y_train, p=cfg.dyn_p, horizon=cfg.horizon)
        stage1_val_ds = RollingXYDataset(x=X_val, y=y_val, p=cfg.dyn_p, horizon=cfg.horizon)
        stage1_train_loader = DataLoader(stage1_train_ds, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers)
        stage1_val_loader = DataLoader(stage1_val_ds, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)

        # Stage 2 needs y0=y[t] for dense increment training.
        stage2_train_ds = RollingXYDatasetWithY0(x=X_train, y=y_train, p=cfg.dyn_p, horizon=cfg.horizon)
        stage2_val_ds = RollingXYDatasetWithY0(x=X_val, y=y_val, p=cfg.dyn_p, horizon=cfg.horizon)
        stage2_train_loader = DataLoader(stage2_train_ds, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers)
        stage2_val_loader = DataLoader(stage2_val_ds, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)

        lavar = LAVAR(
            input_dim=X_train.shape[1],
            latent_dim=cfg.latent_dim,
            transition_order=cfg.dyn_p,
            encoder_hidden_dims=cfg.encoder_hidden,
            decoder_hidden_dims=cfg.decoder_hidden,
        ).to(cfg.device)

        stage1_train_lavar(
            model=lavar,
            train_loader=stage1_train_loader,
            val_loader=stage1_val_loader,
            cfg=cfg,
        )

        # --- Stage 2: density split (dense / sparse / ultra-sparse) ---
        split = stage2_train_supply_density_split(
            lavar=lavar,
            train_loader=stage2_train_loader,
            val_loader=stage2_val_loader,
            cfg=cfg,
        )
        dense_model = split["dense_model"]
        sparse_model = split["sparse_model"]
        ultra_model = split["ultra_model"]

        dense_idx = split["dense_indices"]
        sparse_idx = split["sparse_indices"]
        ultra_idx = split["ultra_indices"]

        # Forecast and stitch back into full Dy.
        with torch.no_grad():
            x_recent_raw = X[t_end - cfg.dyn_p : t_end]
            x_recent = x_scaler.transform(x_recent_raw)

            dev = torch.device(cfg.device)
            x_recent = x_recent.to(dev)

            Dy = y.shape[1]
            y_hat_full = torch.zeros(cfg.horizon, Dy, device=dev, dtype=torch.float32)

            # Dense head predicts increments and needs the current y_t (y0) to integrate.
            y0_raw = y_train_raw[t_end - 1].to(dev)  # (Dy,)

            if dense_model is not None and len(dense_idx) > 0:
                y0_dense = y0_raw[dense_idx]
                y_hat_dense = dense_model(x_recent.unsqueeze(0), y0=y0_dense.unsqueeze(0)).squeeze(0)
                y_hat_full[:, dense_idx] = y_hat_dense

            if sparse_model is not None and len(sparse_idx) > 0:
                y0_sparse = y0_raw[sparse_idx]
                y_hat_sparse = sparse_model(x_recent.unsqueeze(0), y0=y0_sparse.unsqueeze(0)).squeeze(0)
                y_hat_full[:, sparse_idx] = y_hat_sparse

            if ultra_model is not None and len(ultra_idx) > 0:
                y_hat_ultra = ultra_model(x_recent.unsqueeze(0)).squeeze(0)
                y_hat_full[:, ultra_idx] = y_hat_ultra

            y_hat = y_hat_full.detach().cpu()
            y_true = y[t_end : t_end + cfg.horizon].detach().cpu()  # (H, Dy)

            # Store predictions
            times = np.arange(t_end, t_end + cfg.horizon)
            all_pred_times.append(times)
            all_pred_values.append(y_hat.detach().cpu().numpy())
            all_true_values.append(y_true.detach().cpu().numpy())

            # Store error + useful decompositions
            y_hat_cpu = y_hat.detach().cpu()
            y_true_cpu = y_true.detach().cpu()

            err = y_hat_cpu - y_true_cpu            # (H, Dy)
            se = err ** 2                            # (H, Dy)
            fold_mse = se.mean().item()              # scalar over (H * Dy)

            # Axis decompositions
            mse_by_horizon = se.mean(dim=1)          # (H,)
            mse_by_target = se.mean(dim=0)           # (Dy,)

            # Per-fold, per-target error scale (useful for uncertainty bands)
            rmse_by_target = mse_by_target.sqrt()    # (Dy,)
            all_fold_rmse_by_target.append(rmse_by_target.detach().cpu().numpy())

            # Residual decomposition: MSE(e) = Var(e) + Mean(e)^2
            bias_by_target = err.mean(dim=0)         # (Dy,)
            var_by_target = err.var(dim=0, unbiased=False)  # (Dy,)

            # Extra fold diagnostics (more actionable than a single global MSE)
            ae = err.abs()
            fold_mae = ae.mean().item()
            fold_rmse = se.mean().sqrt().item()

            # Simple baseline: "predict last observed value" for all horizons
            y_last = y_train_raw[t_end - 1].detach().cpu()              # (Dy,)
            y_naive = y_last.unsqueeze(0).repeat(cfg.horizon, 1)        # (H, Dy)
            naive_err = y_naive - y_true_cpu
            naive_mse = (naive_err ** 2).mean().item()
            skill_vs_naive = (1.0 - fold_mse / naive_mse) if naive_mse > 0 else float("nan")

            # Where are errors coming from?
            worst_h = int(mse_by_horizon.argmax().item())  # 0-indexed
            worst_h_mse = mse_by_horizon[worst_h].item()

            k = int(min(5, mse_by_target.numel()))
            worst_t_idx = torch.topk(mse_by_target, k=k).indices.tolist()
            worst_targets = ", ".join(
                f"{i}(mse={mse_by_target[i].item():.3g}, bias={bias_by_target[i].item():+.3g})"
                for i in worst_t_idx
            )

            all_fold_errors.append(fold_mse)

        tqdm.write(
            f"Fold {fold_id}/{n_folds} | "
            f"MSE={fold_mse:.4g} RMSE={fold_rmse:.4g} MAE={fold_mae:.4g} | "
            f"naiveMSE={naive_mse:.4g} skill={skill_vs_naive:+.1%} | "
            f"horizon_mean={mse_by_horizon.mean().item():.4g} worst_h={worst_h+1}/{cfg.horizon}({worst_h_mse:.4g}) | "
            f"worst_targets: {worst_targets}"
        )

    except Exception:
        tqdm.write(
            f"\n!!! ERROR in fold {fold_id}/{n_folds} (train_end_day={t_end}, horizon={cfg.horizon}, p={cfg.dyn_p}) !!!\n"
        )
        traceback.print_exc()
        raise


print("Rolling MSE:")
print("mean :", np.mean(all_fold_errors))
print("std  :", np.std(all_fold_errors))


Rolling folds:   0%|          | 0/259 [00:00<?, ?fold/s]


=== Fold 1/259 | train until day 730 ===


Rolling folds:   0%|          | 1/259 [00:26<1:55:50, 26.94s/fold]

Fold 1/259 | MSE=737.5 RMSE=27.16 MAE=8.77 | naiveMSE=746.9 skill=+1.3% | horizon_mean=737.5 worst_h=5/14(2164) | worst_targets: 15(mse=1.42e+04, bias=-105), 28(mse=3.85e+03, bias=-48.9), 21(mse=2.89e+03, bias=-40.9), 19(mse=2.1e+03, bias=-39.7), 9(mse=1.78e+03, bias=-27.6)

=== Fold 2/259 | train until day 737 ===


Rolling folds:   1%|          | 2/259 [00:52<1:51:36, 26.06s/fold]

...

Fold 257/259 | MSE=93.5 RMSE=9.67 MAE=3.29 | naiveMSE=62.63 skill=-49.3% | horizon_mean=93.5 worst_h=5/14(418.9) | worst_targets: 9(mse=812, bias=-15), 19(mse=796, bias=-22.8), 17(mse=572, bias=-14.8), 15(mse=489, bias=-17.6), 28(mse=339, bias=-12.8)

=== Fold 258/259 | train until day 2529 ===


Rolling folds: 100%|█████████▉| 258/259 [3:42:47<01:12, 72.81s/fold]

Fold 258/259 | MSE=88.3 RMSE=9.397 MAE=3.318 | naiveMSE=63.31 skill=-39.5% | horizon_mean=88.3 worst_h=5/14(356.4) | worst_targets: 19(mse=1e+03, bias=-27.1), 9(mse=500, bias=-13.5), 15(mse=491, bias=-18.2), 28(mse=463, bias=-15.6), 17(mse=386, bias=-13.2)

=== Fold 259/259 | train until day 2536 ===


Rolling folds: 100%|██████████| 259/259 [3:44:01<00:00, 51.90s/fold]

Fold 259/259 | MSE=56.35 RMSE=7.507 MAE=2.613 | naiveMSE=46.92 skill=-20.1% | horizon_mean=56.35 worst_h=12/14(305.7) | worst_targets: 9(mse=515, bias=-12.5), 17(mse=327, bias=-11), 15(mse=309, bias=-11.3), 28(mse=305, bias=-8.4), 19(mse=249, bias=-7.09)
Rolling MSE:
mean : 94.59226002196087
std  : 73.4436214521045


In [ ]:
# Draw without confidence intervals
for j in range(y.shape[1]):
    z = 1.95 # ~95% normal interval
    name = supply_cols[j]

    dates = non_supply_df.index  # pandas DatetimeIndex

    # Back-compat: if you didn't rerun the rolling-fold cell, fall back to global fold MSE.
    if "all_fold_rmse_by_target" not in globals():
        Dy = y.shape[1]
        all_fold_rmse_by_target = [np.sqrt(mse) * np.ones(Dy, dtype=float) for mse in all_fold_errors]

    # Stitch rolling forecasts into ONE continuous line:
    # for each timestamp, keep the prediction with the smallest lead time (most recent origin).
    T = len(dates)
    yhat_line = np.full(T, np.nan, dtype=float)
    ci_lo_line = np.full(T, np.nan, dtype=float)
    ci_hi_line = np.full(T, np.nan, dtype=float)
    best_lead = np.full(T, np.inf)

    for times, yhat, rmse_by_target in zip(all_pred_times, all_pred_values, all_fold_rmse_by_target):
        # times: (H,), yhat: (H, Dy), rmse_by_target: (Dy,)
        sigma = float(rmse_by_target[j])
        for lead, idx in enumerate(times):
            idx = int(idx)
            if lead < best_lead[idx]:
                pred = round(yhat[lead, j])
                yhat_line[idx] = pred
                ci_lo_line[idx] = pred - z * sigma
                ci_hi_line[idx] = pred + z * sigma
                best_lead[idx] = lead

    # Start the plot at the first forecast timestamp
    start_idx = int(min(times[0] for times in all_pred_times)) if len(all_pred_times) else 0

    plt.figure(figsize=(12, 5))
    plt.plot(
        dates[start_idx:],
        y[:, j].cpu().numpy()[start_idx:],
        label="y_t (true)",
        linewidth=2,
    )

    plt.plot(
        dates[start_idx:],
        yhat_line[start_idx:],
        label="forecast (stitched)",
        linewidth=1.8,
        color="red",
        alpha=0.7,
    )

    plt.title(f"Supply code {j} (name={name}): true vs forecast")
    plt.xlabel("Time")
    plt.ylabel("Usage")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Draw with confidence intervals
for j in range(y.shape[1]):
    z = 1.95 # ~95% normal interval
    name = supply_cols[j]

    dates = non_supply_df.index  # pandas DatetimeIndex

    # Back-compat: if you didn't rerun the rolling-fold cell, fall back to global fold MSE.
    if "all_fold_rmse_by_target" not in globals():
        Dy = y.shape[1]
        all_fold_rmse_by_target = [np.sqrt(mse) * np.ones(Dy, dtype=float) for mse in all_fold_errors]

    # Stitch rolling forecasts into ONE continuous line:
    # for each timestamp, keep the prediction with the smallest lead time (most recent origin).
    T = len(dates)
    yhat_line = np.full(T, np.nan, dtype=float)
    ci_lo_line = np.full(T, np.nan, dtype=float)
    ci_hi_line = np.full(T, np.nan, dtype=float)
    best_lead = np.full(T, np.inf)

    for times, yhat, rmse_by_target in zip(all_pred_times, all_pred_values, all_fold_rmse_by_target):
        # times: (H,), yhat: (H, Dy), rmse_by_target: (Dy,)
        sigma = float(rmse_by_target[j])
        for lead, idx in enumerate(times):
            idx = int(idx)
            if lead < best_lead[idx]:
                pred = float(yhat[lead, j])
                yhat_line[idx] = pred
                ci_lo_line[idx] = pred - z * sigma
                ci_hi_line[idx] = pred + z * sigma
                best_lead[idx] = lead

    # Start the plot at the first forecast timestamp
    start_idx = int(min(times[0] for times in all_pred_times)) if len(all_pred_times) else 0

    plt.figure(figsize=(12, 5))
    plt.plot(
        dates[start_idx:],
        y[:, j].cpu().numpy()[start_idx:],
        label="y_t (true)",
        linewidth=2,
    )

    mask = (~np.isnan(ci_lo_line[start_idx:])) & (~np.isnan(ci_hi_line[start_idx:]))
    plt.fill_between(
        dates[start_idx:],
        ci_lo_line[start_idx:],
        ci_hi_line[start_idx:],
        where=mask,
        color="red",
        alpha=0.15,
        label="~95% interval (per-fold RMSE)",
    )

    plt.plot(
        dates[start_idx:],
        yhat_line[start_idx:],
        label="forecast (stitched)",
        linewidth=1.8,
        color="red",
        alpha=0.7,
    )

    plt.title(f"Supply code {j} (name={name}): true vs forecast")
    plt.xlabel("Time")
    plt.ylabel("Usage")
    plt.legend()
    plt.tight_layout()
    plt.show()